# Introduction: Grammar of Spatial Data Science

## 1. Learning Objectives

This lecture provides students with **little-to-zero prior knowledge** core competences in Spatial Data Science (SDS).

a. Advance their numerical, computational, and statistical literacy. <br>
b. Introduce basic principles of programming and state of the art (SOTA) computational tools for SDS. <br>
c. Present a rich overview of the methodologies available to the Spatial Data Scientist, as well as intuition as to how and when they can be applied. <br>
d. Focus on real world applications of these techniques in a geographical and applied context.

## 2. Learning Outcomes

By the end of the lecture & workshop, students will be able to:

a. Demonstrate SDS concepts and be able to use tools programmatically to import, manipulate, and analyze spatial data in different formats. <br>
b. Understand the motivation and inner workings of the main methodological approaches of SDS, both analytical and visual. <br>
3. Apply spatial analysis techniques and explain how to interpret the results, in a process of turning data into information.

In [ ]:
# for reproducibility
import random
random.seed(2023)

# for text matching 
import re

# for downloading data and parsing dictionary
import json
import requests

# for spatial data I/O, manipulation, and visualization
import shapely
import leafmap
import osmnx as ox
import networkx as nx
import contextily as ctx
import geopandas as gpd
from shapely.geometry import (
    Point, LineString, Polygon
)
from gadm import GADMDownloader

# standard python libraries
import numpy as np
import pandas as pd
pd.options.display.max_colwidth=200
pd.options.display.max_columns=100
from itertools import combinations
import matplotlib.pyplot as plt

# custom lib for the lecture
# from sds4gdsp import loader, processor

In [ ]:
def make_points(num_points, lng_min, lng_max, lat_min, lat_max):
    coords = []
    for _ in range(num_points):
        lng = random.uniform(lng_min, lng_max)
        lat = random.uniform(lat_min, lat_max)
        coords.append((lng, lat))
    return coords

def make_lines(points):
    random.shuffle(points)
    lines = []
    for orig, dest in zip(points, points[1:]):
        if random.choices([True, False], [30, 70], k=1)[0]:
            lines.append(LineString([orig, dest]))
    return lines

def make_polygon(points):
    return Polygon(points)

def make_spatial_data(n):
    coords = np.random.random((n, 2))
    points = list(map(lambda z: Point(z), coords))
    lines = LineString(points)
    lines = list(map(lambda x: LineString(x), list(zip(points, points[1:]))))
    polygon = Polygon(points)
    return points, lines, polygon

def make_graph(
    origin, network_type, dist=500, dist_type="bbox", retain_all=False, simplify=True
):
    # query the road network using OSMNx
    G = ox.graph_from_point(
        center_point=origin, # origin point of query
        dist=dist, # radius in meters from the origin
        dist_type=dist_type, # examples is `bbox`
        retain_all=retain_all, # filter connected components?
        simplify=simplify, # simplify network topology
        network_type=network_type # filter to <insert type from OSM> roads
    )
    return G

def get_coord_sequence(G, route):
    route_coords = []
    for node in route:
        route_coords.append((G.nodes[node]["x"], G.nodes[node]["y"]))
    return route_coords

## 3. Motivation

### a. What makes SDS so spatial (special)?

a. SDS aims to answer the twofold question -> **WHERE and WHY things happen?**

SDS is interested in understanding both where certain things (or events) happen and why they happen in those places. To do this, Spatial Data Scientists leverage location, distance, and spatial interactions as core aspects of the data they work with and use highly specialized methods and cutting-edge software to analyze, visualize, and draw actionable insights from various spatial use cases.

b. SDS is the intersection of Data Science (DS) and **Geographic Information System (GIS).**

GIS is a niche field that refers to different types of information systems, such as websites, apps, or databases for storing and processing spatial datasets. Today, Spatial Data Scientists use GIS as part of a wider, more modern tech stack allowing them to do more advanced statistical and modeling-based data analyses while also being able to leverage the latest artificial intelligence (AI) and machine learning (ML) techniques more effectively.

c. Only **1 in 3 Data Scientists** are experts in spatial analysis.

SDS is clearly still a niche area of data science, it is nonetheless an attractive area for organizations to explore and focus on in the years ahead. According to this survey **[HI PAO, INSERT LINK PLEASE]**, 9 in 10 of the industries surveyed (telco, software & tech, consulting, cities & government, etc.)
- Have a difficult time finding, recruiting, and hiring talent with SDS in their toolbelt  
- Very likely to increase their investment in SDS for the next two years

d. There's a ton of **publicly available** geo datasets out there!

Yes! That's true. Be it points of interest (POI) like buildings, man-made road networks, internet submarine cables, etc

### Q1: Where (might) have you used geospatial data before?

- Answer #1
- Answer #2
- Answer #n

### b. How can I relate this to my work in Globe?

Geospatial data and assets in the telco universe is **ubiquitous.**

Here are some examples you can encounter in the wild.

**POINT** representation

- Globe cellsites
- Globe fiber NAPs
- Globe physical stores
- Globe sub households
- Globe blitz deployments
- Globe OOH advertisements
- Web-scraped POIs

**POLYGON** representation

- PH admin boundaries
- Globe cellsite coverage areas
- Globe fiber coverage areas
- Custom enumeration units (rectangles, hexagons, etc)
- Competitor coverage estimates
- Building footprints

**GRAPH** representation

- PH road networks
- Globe fiber wiring connections
- WW submarine cable network

### Q2: Can you find the USD 150 million PDSCN project?

The [**Philippine Domestic Submarine Cable Network (PDSCN)**](https://www.globe.com.ph/about-us/newsroom/corporate/pdscn-for-april-2023-completion.html#gref) is the longest undersea fiber cable network in the Philippines to date, spanning a total of 2,500 KM.

In [ ]:
M = leafmap.Map(center=[0, 0], zoom=2)
filepath_cable = "https://raw.githubusercontent.com/opengeos/leafmap/master/examples/data/cable_geo.geojson"
M.add_geojson(filepath_cable, layer_name="WW internet submarine cables")
display(M)

If you have a hard time finding it, get your hands dirty!

1. Load the GeoJSON file using the URL filepath earlier.
2. Parse the dictionary, look for the keyword using regex
3. Use this key to fetch the necessary geometry

In [ ]:
response = requests.get(filepath_cable)
submarine_cables = json.loads(response.text)

# apply 'Philippine' regex to this list, you'll find it for sure!
names_submarine_cables = list(map(lambda x: x["properties"]["name"], submarine_cables["features"]))

r = re.compile(".*Philippine")
key_pdscn = list(filter(r.match, names_submarine_cables))[0]

In [ ]:
key_pdscn

In [ ]:
for test in list(map(lambda x: x["properties"], submarine_cables["features"])):
    if test["name"] == key_pdscn:
        geojson_pdscn = test

In [ ]:
geojson_pdscn # hooray! you can use the coordinate data and plot it to any of your favorite geo library

### Q3: Can you think of any telco geospatial data that is not yet included?

- Answer #1
- Answer #2
- Answer #n

## 4. Familiarizing yourself with **nouns**

### c. Shapely and basic geometric objects

With the use of the **Shapely** library, you can work with geometric objects such as points, lines, and polygons. Most of the data you'll work with can be modeled using these object representations.

In [ ]:
n = 20
points, lines, polygon = make_spatial_data(n)

fig, ax = plt.subplots(1, 3, figsize=(13, 9))
colors = [random.choice(["red", "green", "blue"]) for _ in range(n)]
markersizes = [random.choice(range(100, 600, 100)) for _ in range(n)]
gpd.GeoSeries(points).plot(ax=ax[0], markersize=markersizes, color=colors)
linewidths = [random.choice(range(2, 10, 2)) for _ in range(n)]
gpd.GeoSeries(lines).plot(ax=ax[1], linewidth=linewidths, color=colors)
gpd.GeoSeries(polygon).plot(ax=ax[2], linewidth=5, color=random.choice(colors))
ax[0].set_title(f"shapely.geometry.Point", fontsize=18)
ax[1].set_title(f"shapely.geometry.LineString", fontsize=18)
ax[2].set_title(f"shapely.geometry.Polygon", fontsize=18)
ax[0].tick_params(axis="both", which="major", labelsize=12)
ax[0].tick_params(axis="both", which="minor", labelsize=12)
ax[1].tick_params(axis="both", which="major", labelsize=12)
ax[1].tick_params(axis="both", which="minor", labelsize=12)
ax[2].tick_params(axis="both", which="major", labelsize=12)
ax[2].tick_params(axis="both", which="minor", labelsize=12)
plt.tight_layout();

In [ ]:
help(points[0])

In [ ]:
lines[0]

In [ ]:
polygon

### Q4: Provide real-world scenario examples of a point, linestring and polygon
- Answer for point : 
- Answer for linestring : 
- Answer for polygon : 

### Nice! To sum it up:
- A **point** generally would represent a single location in Earth defined by a pair of coordinates
- A **linestring** is a sequence of connected points and represents a continuous path. This is made up of at least two coordiantes
- A **polygon** is closed 2D shape usually defined by a sequence of coordinates that form a closed loop

### d. Geopandas and dataframes

This tool allows you to process tabular geometric data with a Pandas API like interface. Let's look at an example using the GADM shapefile (PH).

In [ ]:
country_name = "Philippines"
gadm_version = "4.0"
ad_level = 2
downloader = GADMDownloader(version=gadm_version)
gadm = downloader.get_shape_data_by_country_name(
    country_name=country_name, ad_level=ad_level
)

In [ ]:
gadm.head()

Our dataset contains the city/town level boundaries (L2 in GADM). Let's look at the dtype of one geom object.

In [ ]:
print(type(gadm.loc[0, "geometry"]))
gadm.loc[0, "geometry"]

Like pandas, we can perform `.plot()` in geopandas. Let's look into the PH boundaries.

In [ ]:
fig, ax = plt.subplots(1, figsize=(7, 10))
gadm.plot(ax=ax)
ax.axis("off");

In [ ]:
is_metro_manila = gadm.NAME_1=="Metropolitan Manila"
gadm_metro_manila = gadm.loc[is_metro_manila].reset_index()
display(gadm_metro_manila)

In [ ]:
for idx, row in gadm_metro_manila.iterrows():
    display(row.NAME_2)
    display(row.geometry)

### Q5: Which two towns in metro manila has the smallest and largest land mass, respectively?

Show your solution.

In [ ]:
gadm_metro_manila.iloc[[np.argmin(gadm_metro_manila.to_crs(3857).area)]]

In [ ]:
gadm_metro_manila.iloc[[np.argmax(gadm_metro_manila.to_crs(3857).area)]]

### e. OSMNx and graphs

This is a useful library for dealing with spatial graph network datasets. One good example of this is the street networks we see when travelling in land with a vehicle everyday. You can use this to model, analyze, and visualize street networks and other geospatial features from OSM. Let us walk thru an example.

In [ ]:
COORD_TGT = (14.553514, 121.050110) # the globe tower @ bonifacio global city, taguig
COORD = COORD_TGT # you can replace this with any point within the region of interest
NAME_PLACE = "TGT"
G_drive = make_graph(origin=COORD, network_type="drive")
G_walk = make_graph(origin=COORD, network_type="walk")
G_bike = make_graph(origin=COORD, network_type="bike")

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(30, 30))
ox.plot_graph(G=G_drive, ax=ax[0], node_size=50, edge_linewidth=2, node_color="red", bgcolor="white", show=False, close=False)
ox.plot_graph(G=G_bike, ax=ax[1], node_size=50, edge_linewidth=2, node_color="red", bgcolor="white", show=False, close=False)
ox.plot_graph(G=G_walk, ax=ax[2], node_size=50, edge_linewidth=2, node_color="red", bgcolor="white", show=False, close=False)
ax[0].set_title(f"driving around {NAME_PLACE}?\n", fontsize=18)
ax[1].set_title(f"biking around {NAME_PLACE}?\n", fontsize=18)
ax[2].set_title(f"walking around {NAME_PLACE}?\n", fontsize=18)
ax[0].scatter(*COORD[::-1], c="blue", s=250)
ax[1].scatter(*COORD[::-1], c="blue", s=250)
ax[2].scatter(*COORD[::-1], c="blue", s=250);

- Walkable networks are denser than the driving networks
- Road networks like this can (and should) change over time

`NOTES`

These data representations isn't exclusive to Shapely, GeoPandas, and OSMNx. For instance, Snowflake (the data cloud company) has [its own](https://docs.snowflake.com/en/sql-reference/data-types-geospatial) geospatial data types and formats patterned to PostGIS. The more you work with SDS and understand its nuts and bolts, the more you realize that beyond these vast offerings of tools and software, there are common concepts and abstractions that needs to be mastered. Do yourself a favor and invest some time on the core concepts as well instead of purely memorizing the syntax of these modern tools.

### Q6: How large is each of the road networks displayed above?

Hint: Look at all the properties/attributes of the three G's.

- Answer for G_drive
- Answer for G_bike
- Answer for G_walk

## 5. Getting things done with **verbs**

### Buffering points and spatial matching

One of the common task in SDS is spatial matching. Say for example, you are given the task to find out **how many points of interest are within x kilometers from this origin point?**

In [ ]:
n = 40
points = make_spatial_data(n)[0]

fig, ax = plt.subplots(1, figsize=(7, 12))
gs_points = gpd.GeoSeries(points)
gs_points.plot(ax=ax, zorder=2, color="red")
mean_x = np.mean(list(map(lambda z: z.x, points)))
mean_y = np.mean(list(map(lambda z: z.y, points)))
mean_point = Point(mean_x, mean_y)
mean_point_buffered = gpd.GeoSeries(mean_point).buffer(3e-1)
bmask_within = [mean_point_buffered.contains(p)[0] for p in gs_points]
mean_point_buffered.plot(ax=ax, alpha=0.5, zorder=1, color="green")
gs_points.loc[bmask_within].plot(ax=ax, zorder=3, color="blue")
ax.legend(["outside", "inside"]);

### Q7: What if a rectangular buffer is needed? How many points will intersect above?

Hint: Look at shapely's [docs](https://shapely.readthedocs.io/en/stable/manual.html) and find something called **cap_style**.

- Answer for rectangular cap style

### Identifying the shortest path bet two points

In [ ]:
COORD_UPT = (14.556714, 121.054081)
orig_coord = COORD_TGT
dest_coord = COORD_UPT

orig_node = ox.distance.nearest_nodes(G_bike, *orig_coord[::-1])
dest_node = ox.distance.nearest_nodes(G_bike, *dest_coord[::-1])
route_od = ox.shortest_path(G_bike, orig_node, dest_node, weight="travel_time")
route_do = ox.shortest_path(G_bike, dest_node, orig_node, weight="travel_time")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
ox.plot_graph(G=G_bike, ax=ax, node_size=50, edge_linewidth=2, node_color="red", bgcolor="white", show=False, close=False)
gpd.GeoSeries(shapely.geometry.LineString(get_coord_sequence(G_bike, route_od))).plot(ax=ax, color="green", linewidth=10)
gpd.GeoSeries(shapely.geometry.LineString(get_coord_sequence(G_bike, route_do))).plot(ax=ax, color="blue", linewidth=10)
ax.scatter(*COORD_TGT[::-1], c="green", s=200)
ax.scatter(*COORD_UPT[::-1], c="blue", s=200)
ax.legend(["__", "__", "O/D shortest path", "D/O shortest path", "origin", "dest"], edgecolor="white", loc="upper left");

`NOTES`

The shortest O-D path is not always the same D-O path in real-life! Most especially in driveable networks.

In [ ]:
nodes_walk, edges_walk = ox.graph_to_gdfs(G_walk)
print(f"nodes_walk: {type(nodes_walk)}\n", f"edges_walk: {type(nodes_walk)}")

In [ ]:
nodes_walk.head()

In [ ]:
edges_walk.head()

## 6. References [WIP]

1. https://carto.com/what-is-spatial-data-science

### 7. TODO [WIP]

1. Expound on the solution per mini quiz, like CRS conversion
2. Explain the difference between pandas and geopandas